## 概述
命名实体-描述实体的词汇，具有以下共性：
- 数量无穷。
- 构词灵活。
- 类别模糊。

识别出文本中命名实体的边界与类别的任务称为命名实体识别。

## 基于规则的命名实体识别
### 基于规则的音译人名识别
其逻辑如下：
1. 若粗分结果中某词语的备选词性含有 nrf 则触发规则 2；
2. 从该词语出发从左往右扫描，若遇到音译人名库中的词语，则合并

### 基于规则的日本人名识别
1. 文本中匹配日本人名的姓氏和名字，记作x和m
2. 合并连续的xm为日本人名

### 基于规则的数词英文识别

## 基于层叠隐马尔可夫模型的角色标注框架
### 基于角色标注的中国人名识别
中国科学院计算技术研究所软件实验室张华平和刘群教授在《基于角色标注的中国人名自动识别研究》
- 1. 姓氏 - B
- 2. 双名的首字 - C
- 3. 双名的末字 - D
- 4. 单名 - E
- 5. 前缀 - F
 ……
 
### 基于角色标注的地名识别
- 1. 地名的上文 - A
- 2. 地名的下文 - B

### 基于角色标注的机构名称
- 1. 上文 - A
- 2. 下文 - B

## 基于序列标注的命名实体识别
1. hmm 
2. crf
3. perceptron

## 实现借口
### 基于隐马尔可夫模型序列标注的命名实体识别
https://github.com/hankcs/pyhanlp/blob/master/tests/book/ch08/demo_hmm_ner.py

In [4]:
import os
import zipfile

from pyhanlp import *
from pyhanlp.static import download, remove_file, HANLP_DATA_PATH
# from tests.book.ch07 import pku # 如下

In [6]:
# test_utility.py
def test_data_path():
    """获取测试数据路径"""
    data_path = os.path.join(HANLP_DATA_PATH, 'test')
    if not os.path.isdir(data_path):
        os.mkdir(data_path)
    return data_path


def ensure_data(data_name, data_url: str):
    """目标文件下载模块"""
    root_path = test_data_path()
    dest_path = os.path.join(root_path, data_name)
    # 如果dest_path文件存在,直接返回文件路径
    if os.path.exists(dest_path):
        return dest_path
    
    if data_url.endswith('.zip'):
        dest_path += '.zip'
    # 文件不存在时 下载数据到指定目录
    download(data_url, dest_path)
    # 解压文件
    if data_url.endswith('.zip'):
        with zipfile.ZipFile(dest_path, 'r') as archive:
            archive.extractall(root_path)
        remove_file(dest_path)   # 删除压缩包
        dest_path = dest_path[:-4]
    return dest_path


In [10]:
# pku.py  # 加载训练语料的路径
# 以后使用hmm分词时，训练语料库的类型与此相同即可
# 本目录下的`199801.txt`在原版的基础上做了如下修改：
# 1. 为了符合习惯，姓+名合并为姓名
# 2. 格式升级为兼容2014版，复合词中括号后添加“/”
# 3. 文本编码调整为UTF-8
PKU98 = ensure_data('pku98', "http://file.hankcs.com/corpus/pku98.zip")
PKU199801 = os.path.join(PKU98, '199801.txt')
PKU199801_TRAIN = os.path.join(PKU98, '199801-train.txt')
PKU199801_TEST = os.path.join(PKU98, '199801-test.txt')
POS_MODEL = os.path.join(PKU98, 'pos.bin')
NER_MODEL = os.path.join(PKU98, 'ner.bin')

In [12]:
HMMNERecognizer = JClass('com.hankcs.hanlp.model.hmm.HMMNERecognizer')
AbstractLexicalAnalyzer = JClass(
    'com.hankcs.hanlp.tokenizer.lexical.AbstractLexicalAnalyzer')
PerceptronSegmenter = JClass('com.hankcs.hanlp.model.perceptron.PerceptronSegmenter')
PerceptronPOSTagger = JClass('com.hankcs.hanlp.model.perceptron.PerceptronPOSTagger')
Utility = JClass('com.hankcs.hanlp.model.perceptron.utility.Utility')

In [37]:
def train(corpus):
    recognizer = HMMNERecognizer()
    recognizer.train(corpus)
    return recognizer

def test(recognizer):
    word_array = ["华南", "电力", "公司"]  # 构造单词序列
    pos_array = ["ns", "n", "n"]  # 构造词性序列
    ner_array = recognizer.recognize(word_array, pos_array)
    for word, tag, ner in zip(word_array, pos_array, ner_array):
        print(f'{word}\t{tag}\t{ner}')
    analyzer = AbstractLexicalAnalyzer(
        PerceptronSegmenter(), PerceptronPOSTagger(), recognizer)
    print(analyzer.analyze("华南电力公司董事长刘良栋和秘书章梦丹来到美国纽约现代艺术博物馆参观"))
    scores = Utility.evaluateNER(recognizer, PKU199801_TEST)
    print(scores)
    Utility.printNERScore(scores)

In [39]:
recognizer = train(PKU199801_TRAIN)
test(recognizer)

华南	ns	B-nt
电力	n	M-nt
公司	n	E-nt
[华南/ns 电力/n 公司/n]/nt 董事长/n 刘良栋/nr 和/c 秘书/n 章梦丹/nr 来到/v 美国纽约/ns 现代/ntc 艺术/n 博物馆/n 参观/v
{avg.=[D@396a51ab, ns=[D@51081592, nt=[D@7f9a81e8}


### 基于感知机序列标注的命名实体识别